##### Импорты

In [2]:
import random
import math

### Алгорим RSA

In [3]:
# def inverseMul(e, phi):
#     d = 0
#     x1 = 0
#     x2 = 1
#     y1 = 1
#     tPhi = phi
    
#     while e > 0:
#         temp1 = tPhi // e
#         temp2 = tPhi - temp1 * e
#         tPhi = e
#         e = temp2
        
#         x = x2 - temp1 * x1
#         x2 = x1
#         x1 = x
        
#         d = y1
#         y1 = x2 - temp1 * y1
        
#     if tPhi == 1:
#         return d + phi

def isPrime(n):
    if n <= 1 or (n % 2 == 0 and n > 2): 
        return False
    return all(n % i for i in range(3, int(math.sqrt(n)) + 1, 2))

def getPrimeNumber(start=1, end=100):
    while True:
        p = random.randint(start, end)
        if isPrime(p):
            return p

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def getKeyPair(p, q):
    keyGeneration = 'Генерация ключей:\n'
    keyGeneration += f'p: {p:<6} q: {q:<6}\n'
    n = p * q
    keyGeneration += f'n = (p * q) = {n:<11}\n'
    phi = (p-1) * (q-1)
    keyGeneration += f'ψ = (p - 1) * (q - 1) = {phi:<11}\n'

    e = random.randrange(1, phi)
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    keyGeneration += f'e = {e:<11}\n'
    d = pow(e, -1, phi)
    keyGeneration += f'd = (e ^ -1 mod ψ) = {d}\n'
    keyGeneration += f'Открытый ключ: ({e}, {n})\n'
    keyGeneration += f'Закрытый ключ: ({d}, {n})\n'

    return (e, n), (d, n), keyGeneration

def rsaEncrypt(text, key):
    result = []
    encryptionData = 'Шифрование:\n'
    e, n = key
    encryptionData += 'Символ | Номер | Значение\n'
    encryptionData += '-' * 27
    for char in text:
        charNum = ord(char)
        val = pow(charNum, e, n)
        result.append(val)
        encryptionData += f'\n{char:^6} | {charNum:^5} | {val}'

    return result, encryptionData

def rsaDecrypt(message, key):
    result = ''
    decryptionData = 'Расшифровка:\n'
    d, n = key
    decryptionData += '  Значение  | Номер | Символ\n'
    decryptionData += '-' * 32
    for number in message:
        charNum = pow(number, d, n)
        char = chr(charNum)
        result += char
        decryptionData += f'\n{number:^12}| {charNum:^6}| {char:^7}'
    return result, decryptionData

public, private, keyGenerationLog = getKeyPair(getPrimeNumber(10000, 100000), getPrimeNumber(10000, 100000))
text = "Isyuk"
encrypt, encryptionLog = rsaEncrypt(text, public)
decrypt, decryptionLog = rsaDecrypt(encrypt, private)

encrypt = ' '.join(str(val) for val in encrypt)

print(f'''{keyGenerationLog}
Исходный текст:      {text}
{encryptionLog}
Зашифрованный текст: {encrypt}

{decryptionLog}
Дешифрованный текст: {decrypt}
''')

Генерация ключей:
p: 28559  q: 60623 
n = (p * q) = 1731332257 
ψ = (p - 1) * (q - 1) = 1731243076 
e = 1202477765 
d = (e ^ -1 mod ψ) = 443322521
Открытый ключ: (1202477765, 1731332257)
Закрытый ключ: (443322521, 1731332257)

Исходный текст:      Isyuk
Шифрование:
Символ | Номер | Значение
---------------------------
  I    |  73   | 1543793115
  s    |  115  | 366580116
  y    |  121  | 245649677
  u    |  117  | 1388072964
  k    |  107  | 1155247364
Зашифрованный текст: 1543793115 366580116 245649677 1388072964 1155247364

Расшифровка:
  Значение  | Номер | Символ
--------------------------------
 1543793115 |   73  |    I   
 366580116  |  115  |    s   
 245649677  |  121  |    y   
 1388072964 |  117  |    u   
 1155247364 |  107  |    k   
Дешифрованный текст: Isyuk



### Алгоритм на основе задачи об укладке ранца

In [8]:
def getSuperIncreasingSequence(n, start=1, end=10):
    result = []
    total = 0
    for _ in range(n):
        next = random.randint(total + start, total + end)
        result.append(next)
        total += next
    return result


def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def getCoprimeNumber(num):
    for i in range(2, num):
        if gcd(num, i) == 1:
            return i


def getIndecies(byte):
    return [i for i, x in enumerate(byte) if x == '1']


def getByteFromIndecies(indecies):
    byte = ['0' for i in range(8)]
    for index in indecies:
        byte[index] = '1'
    return ''.join([bit for bit in byte])


def packBackpack(weights, capacity):
    n = len(weights)
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    keep = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for w in range(1, capacity + 1):
            if weights[i-1] <= w:
                if weights[i-1] + dp[i-1][w-weights[i-1]] > dp[i-1][w]:
                    dp[i][w] = weights[i-1] + dp[i-1][w-weights[i-1]]
                    keep[i][w] = 1
                else:
                    dp[i][w] = dp[i-1][w]
            else:
                dp[i][w] = dp[i-1][w]

    K = capacity
    idx = []
    for i in range(n, 0, -1):
        if keep[i][K] == 1:
            idx.append(i-1)
            K -= weights[i-1]

    return list(reversed(idx))


def backpackPackingProblemEncrypt(text, key):
    result = []
    encryptionData = 'Шифрование:\n'
    srcValue = [bin(c)[2:] for c in text.encode('cp1251')]
    indecies = [getIndecies(val) for val in srcValue]
    for ls in indecies:
        val = 0
        for index in ls:
            val += key[index]
        result.append(val)

    encryptionData += 'Символ | bin-code |               Сумма весов                | Результат\n'
    encryptionData += '-' * 72
    for i in range(len(text)):
        sum = ' + '.join([str(key[index]) for index in indecies[i]])
        encryptionData += f'\n{text[i]:^6} | {srcValue[i]} | {sum:^40} | {result[i]:>6}'

    return result, encryptionData


def backpackPackingProblemDecrypt(message, key, mod, n):
    result = ''
    decryptionData = 'Расшифровка:\n'
    n1 = pow(n, -1, mod)
    decryptionData += f'n^-1 = {n1}\n'
    decryptionData += f'({n} * {n1}) mod {mod} = {(n * n1) % mod}\n'
    decryptionData += 'Шифр | Вес  |               Сумма весов                | bin-code | Символ\n'
    decryptionData += '-' * 74
    for val in message:
        d = (val * n1) % mod
        indecies = packBackpack(key, d)
        byte = int(getByteFromIndecies(indecies), 2)
        bytesData = bytes([byte])
        char = bytesData = bytesData.decode('cp1251')
        result += char

        sumStr = ' + '.join([str(key[index]) for index in indecies])
        decryptionData += f'\n{val:>4} | {d:>4} | {sumStr:^40} | {bin(byte)[2:]} | {char:^4}'

    return result, decryptionData


private = getSuperIncreasingSequence(10)
mod = random.randint(sum(private), sum(private) + 10)
n = getCoprimeNumber(mod)
public = [(n*val) % mod for val in private]


text = 'Исюк'

encrypt, encryptionLog = backpackPackingProblemEncrypt(text, public)
decrypt, decryptionLog = backpackPackingProblemDecrypt(
    encrypt, private, mod, n)

encrypt = ' '.join([str(val) for val in encrypt])

print(f'''Закрытый ключ:
{private}
Открытый ключ:
{public}

Исходный текст:      {text}
{encryptionLog}
Зашифрованный текст: {encrypt}

{decryptionLog}
Дешифрованный текст: {decrypt}
''')

Закрытый ключ:
[6, 16, 29, 53, 105, 215, 434, 861, 1724, 3450]
Открытый ключ:
[18, 48, 87, 159, 315, 645, 1302, 2583, 5172, 3448]

Исходный текст:      Исюк
Шифрование:
Символ | bin-code |               Сумма весов                | Результат
------------------------------------------------------------------------
  И    | 11001000 |              18 + 48 + 315               |    381
  с    | 11110001 |        18 + 48 + 87 + 159 + 2583         |   2895
  ю    | 11111110 |  18 + 48 + 87 + 159 + 315 + 645 + 1302   |   2574
  к    | 11101010 |        18 + 48 + 87 + 315 + 1302         |   1770
Зашифрованный текст: 381 2895 2574 1770

Расшифровка:
n^-1 = 2301
(3 * 2301) mod 6902 = 1
Шифр | Вес  |               Сумма весов                | bin-code | Символ
--------------------------------------------------------------------------
 381 |  127 |               6 + 16 + 105               | 11001000 |  И  
2895 |  965 |          6 + 16 + 29 + 53 + 861          | 11110001 |  с  
2574 |  858 |    6 

### Алгоритм шифрования Эль-Гамаля

In [110]:
def isPrime(n):
    if n <= 1 or (n % 2 == 0 and n > 2): 
        return False
    return all(n % i for i in range(3, int(math.sqrt(n)) + 1, 2))

def getPrimeNumber(start = 1, end = 100):
    while True:
        prime = random.randint(start, end)
        if isPrime(prime):
            return prime

def getKeys(prime, generator):
    private = random.randint(1, prime - 1)
    public = pow(generator, private, prime)
    return private, public

def getGenerator(prime):
    for generator in range(2, prime):
        if all(pow(generator, i, prime) != 1 for i in range(1, prime - 1)):
            return generator

def elGamalCipherEncrypt(text, prime, key, gen):
    result = []
    encryptionData = 'Шифрование:\n'
    encryptionData += 'Символ | Код  | Случайное k | Первая часть (a) | Вторая часть (b)\n'
    encryptionData += '-' * 65
    for char in text:
        charNum = ord(char)
        k = random.randint(1, prime - 1)
        a = pow(gen, k, prime)
        b = (pow(key, k, prime) * charNum) % prime
        result.append((a, b))

        encryptionData += f'\n{char:^7}| {charNum:^5}| {k:^12}| {a:^17}| {b:^15}'

    return result, encryptionData

def elGamalCipherDecrypt(message, prime, key):
    result = ''
    decryptionData = 'Расшифровка:\n'
    decryptionData += 'Первая часть (а) | Вторая часть (b) | a^(p-x-1) mod p | Код  | Символ\n'
    decryptionData += '-' * 69
    for item in message:
        a = item[0]
        b = item[1]
        a1 = pow(a, prime - key - 1, prime)
        m = (b * a1) % prime
        result += chr(m)

        decryptionData += f'\n{a:^17}| {b:^17}| {a1:^16}| {m:^5}| {chr(m):^7}'

    return result, decryptionData



text = 'Исюк'

prime = getPrimeNumber(10000, 100000)
generator = getGenerator(prime)
private, public = getKeys(prime, generator)

encrypt, encryptionLog = elGamalCipherEncrypt(text, prime, public, generator)
decrypt, decryptionLog = elGamalCipherDecrypt(encrypt, prime, private)

encrypt = ' '.join([f'{val[0]} {val[1]}' for val in encrypt])

print(f'''Открытый ключ:
   p  |  g  |   y
{prime:^5} | {generator:^3} | {public:^5}

Закрытый ключ:
   p  |  g  |   x
{prime:^5} | {generator:^3} | {private:^5}

Исходный текст:      {text}
{encryptionLog}
Зашифрованный текст: {encrypt}

{decryptionLog}
Дешифрованный текст: {decrypt}
''')



Открытый ключ:
   p  |  g  |   y
55837 |  5  | 4287 

Закрытый ключ:
   p  |  g  |   x
55837 |  5  | 20552

Исходный текст:      Исюк
Шифрование:
Символ | Код  | Случайное k | Первая часть (a) | Вторая часть (b)
-----------------------------------------------------------------
   И   | 1048 |    51196    |       23954      |      18476     
   с   | 1089 |    28527    |       9837       |      29513     
   ю   | 1102 |     8393    |       15401      |      15434     
   к   | 1082 |    19264    |       1054       |      10893     
Зашифрованный текст: 23954 18476 9837 29513 15401 15434 1054 10893

Расшифровка:
Первая часть (а) | Вторая часть (b) | a^(p-x-1) mod p | Код  | Символ
---------------------------------------------------------------------
      23954      |       18476      |       1221      | 1048 |    И   
      9837       |       29513      |      32778      | 1089 |    с   
      15401      |       15434      |      26135      | 1102 |    ю   
      1054       |       108